#1. Carregamento e Seleção de Dados
## Primeiro carregamos o dataset e selecionamos 10 frases para demonstração

In [65]:
import pandas as pd
import numpy as np
import re
from nltk.stem import RSLPStemmer
import nltk

nltk.download('punkt_tab')
nltk.download('rslp')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [66]:


# ID da  planilha no Google Sheets
sheet_id = "1SNV2AooYAxRZO0ubQXdWc6uCnxqk9axb"

# URL de exportação direta para XLSX
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx"

try:
    # Carrega diretamente do Google Sheets
    df = pd.read_excel(url)
    print("Arquivo carregado com sucesso a partir do link do Google Sheets!")
except Exception as e:
    print(f"Ocorreu um erro ao carregar o arquivo: {e}")
    raise
df = df.dropna(subset=['Notícia'])  # Remover linhas com texto faltante
amostras = df.sample(10, random_state=42)['Notícia'].tolist()  # 10 frases aleatórias

print("Exemplos brutos:")
for i, texto in enumerate(amostras[:3], 1):  # Mostrar 3 primeiras para ilustração
    print(f"\nTexto {i} original:\n{texto}")


Arquivo carregado com sucesso a partir do link do Google Sheets!
Exemplos brutos:

Texto 1 original:
A Nike lançou a Phantom Luna, a chuteira feminina, às vésperas do amistoso da Seleção Brasileira Feminina de Futebol que bateu o Chile por 4 x 0 neste domingo (2), em Brasília (DF).

A chuteira marca o protesto da super atacante brasileira Marta, que fez campanha na Copa do Mundo Feminina de 2019, pela visibilidade do torneio e igualdade com os atletas masculinos.

Sete vezes eleita a melhor jogadora do mundo, Marta transformou a chuteira preta, sem detalhes, em símbolo colorido de igualdade de gênero no futebol. No Mundial da Austrália e da Nova Zelândia, entre julho e agosto, as jogadoras patrocinadas pela Nike usarão, pela primeira vez,  a chuteira.

Comercial
A chuteira

A Nike, que também responsável por fornecer os uniformes da Seleção Brasileira, informou que, desde a pesquisa até o design, passando pela etapa de testes, as jogadoras estiveram sempre no centro do processo.

Com a

# 2. Técnica 1: Tokenização (usando biblioteca)
# **Objetivo:** Dividir textos em unidades significativas (palavras/pontuação)

In [67]:
# 2. Técnica 1: Tokenização (usando biblioteca)
# **Objetivo:** Dividir textos em unidades significativas (palavras/pontuação)

# %%
from nltk.tokenize import word_tokenize

def tokenizacao(texto):
    return word_tokenize(texto.lower(), language='portuguese')

# Aplicar em todos os textos
tokens = [tokenizacao(texto) for texto in amostras]

print("\nExemplo de tokenização:")
print(f"Antes: {amostras[0][:50]}...")
print(f"Depois: {tokens[0][:5]}...")  # Mostrar primeiros 5 tokens



Exemplo de tokenização:
Antes: A Nike lançou a Phantom Luna, a chuteira feminina,...
Depois: ['a', 'nike', 'lançou', 'a', 'phantom']...


# 3. Técnica 2: Remoção de Stopwords (implementação manual)
# **Objetivo:** Remover palavras comuns que não agregam significado


# Lista customizada criada manualmente (implementação from scratch)
stopwords_personalizadas = [
    'de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'para', 'com', 'uma', 'os',
    'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele'
]


In [68]:
stopwords_personalizadas = [
    'de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'para', 'com', 'uma', 'os',
    'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele'
]

def remover_stopwords(tokens):
    return [token for token in tokens if token not in stopwords_personalizadas]

# Aplicar filtro
tokens_sem_stopwords = [remover_stopwords(doc) for doc in tokens]

print("\nExemplo de remoção de stopwords:")
print(f"Antes: {tokens[0][:10]}...")
print(f"Depois: {tokens_sem_stopwords[0][:10]}...")


Exemplo de remoção de stopwords:
Antes: ['a', 'nike', 'lançou', 'a', 'phantom', 'luna', ',', 'a', 'chuteira', 'feminina']...
Depois: ['nike', 'lançou', 'phantom', 'luna', ',', 'chuteira', 'feminina', ',', 'às', 'vésperas']...


# 4. Técnica 3: Radicalização (Stemming)
## **Objetivo:** Reduzir palavras à sua forma base

In [69]:
stemmer = RSLPStemmer()

def aplicar_stemming(tokens):
    return [stemmer.stem(token) for token in tokens]

# Processar textos
tokens_stemizados = [aplicar_stemming(doc) for doc in tokens_sem_stopwords]

print("\nExemplo de stemming:")
print(f"Antes: {tokens_sem_stopwords[0][:10]}...")
print(f"Depois: {tokens_stemizados[0][:10]}...")


Exemplo de stemming:
Antes: ['nike', 'lançou', 'phantom', 'luna', ',', 'chuteira', 'feminina', ',', 'às', 'vésperas']...
Depois: ['nik', 'lanç', 'phantom', 'lun', ',', 'chut', 'feminin', ',', 'às', 'vésp']...


# 5. Pipeline Completo de Pré-processamento
# Combinação de todas as etapas em um fluxo automatizado


In [70]:
def pipeline_preprocessamento(texto):
    # Etapas sequenciais
    tokens = tokenizacao(texto)
    tokens = remover_stopwords(tokens)
    tokens = aplicar_stemming(tokens)
    return tokens

# Aplicar em toda a amostra
textos_processados = [pipeline_preprocessamento(texto) for texto in amostras]

In [71]:
# 6. Resultados Finais
# Comparação entre textos originais e pré-processados

# %%
print("\nComparação final:")
for i in range(3):  # Mostrar 3 exemplos
    print(f"\nOriginal ({i+1}):\n{amostras[i][:70]}...")
    print(f"Pré-processado: {textos_processados[i][:10]}...")


Comparação final:

Original (1):
A Nike lançou a Phantom Luna, a chuteira feminina, às vésperas do amis...
Pré-processado: ['nik', 'lanç', 'phantom', 'lun', ',', 'chut', 'feminin', ',', 'às', 'vésp']...

Original (2):
A 6ª câmara do Conselho de Ética do Conselho Nacional de Autorregulame...
Pré-processado: ['6ª', 'câm', 'conselh', 'étic', 'conselh', 'nacion', 'autorregulament', 'publicitár', 'julg', 'últ']...

Original (3):
A Libbs Farmacêutica abre, na quarta-feira (9), as inscrições para a q...
Pré-processado: ['libb', 'farmac', 'abr', ',', 'quarta-f', '(', '9', ')', ',', 'inscr']...



## Conclusão do Colab: Pré-processamento de Textos para Análise de Sentimentos

O notebook estruturou um pipeline de pré-processamento de textos utilizando três técnicas fundamentais para análise de sentimentos em português: **tokenização**, **remoção manual de stopwords** e **stemming**. O processo foi aplicado sobre 10 frases selecionadas aleatoriamente da base de dados do projeto, garantindo diversidade e evitando sobreposição com outros grupos.


### Resumo das Técnicas Utilizadas

- **Tokenização:**  
  Os textos foram divididos em tokens (palavras e pontuações), facilitando a manipulação e análise subsequente. O uso de uma biblioteca especializada (NLTK) garantiu precisão na segmentação linguística, essencial para qualquer pipeline de PLN.

- **Remoção de Stopwords (implementação manual):**  
  Uma lista customizada de palavras comuns foi criada e aplicada sem o uso de bibliotecas prontas, atendendo ao requisito de implementação "from scratch". Isso removeu termos que não agregam significado, tornando o texto mais relevante para análise sem ruídos.

- **Stemming:**  
  Utilizando o RSLPStemmer da NLTK, as palavras foram reduzidas à sua forma radical, agrupando diferentes variações de um mesmo termo e melhorando a generalização dos modelos de machine learning.


### Funcionamento do Pipeline

O pipeline sequencial automatiza as três etapas, permitindo o pré-processamento consistente de qualquer texto da base. O resultado é um conjunto de tokens representativos, livres de ruídos e prontos para alimentar modelos de análise de sentimentos ou outras tarefas de PLN.


### Resultados Observados

A comparação entre os textos originais e os pré-processados demonstra a eficácia das técnicas:

- Redução significativa de palavras irrelevantes.
- Normalização dos termos, facilitando a identificação de padrões e sentimentos.
- Preparação dos dados para algoritmos de aprendizado de máquina, com menor dimensionalidade e maior qualidade semântica.


### Próximos Passos Sugeridos

- Ampliar a lista de stopwords para abranger termos específicos do domínio do projeto.
- Testar técnicas mais avançadas, como lematização, para maior precisão na normalização.
- Incluir tratamento de negações e elementos não textuais (como emojis), que podem ser relevantes para análise de sentimentos em contextos informais.